In [13]:
%pip install azure-storage-file-datalake

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
pip install beautifulsoup4


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Scrapping

In [6]:
import pandas as pd
import requests
import io # Importamos la biblioteca io para manejar strings como archivos

# URL de la página de Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_association_football_stadiums_by_capacity"

# 🛡️ Definir el User-Agent para simular un navegador y evitar el 403
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

print("Paso 1: Realizando la solicitud HTTP con User-Agent...")
try:
    # 1. Hacemos la solicitud GET con los headers
    response = requests.get(url, headers=headers)
    response.raise_for_status() # Lanza un error si el status code es 4xx o 5xx
    
    # 2. Leemos el contenido de la respuesta y lo envolvemos en io.StringIO
    # Esto permite a Pandas tratar la cadena HTML como un archivo.
    html_content = io.StringIO(response.text)
    
    print("Paso 2: Solicitud exitosa (Status 200). Leyendo tablas...")

    # 3. Usamos pd.read_html para leer el contenido del "archivo" en memoria
    # El argumento 'match' ayuda a encontrar la tabla principal.
    tables = pd.read_html(html_content, match='Stadium', header=0) 
    
    # La tabla de estadios es la primera que contiene la columna "Stadium".
    df_stadiums = tables[0]

    print("✅ ¡DataFrame extraído con éxito!")
    
    # Mostrar las primeras 5 filas y la información de las columnas
    print("\n--- Primeras 5 filas ---")
    print(df_stadiums.head())
    print("\n--- Información del DataFrame ---")
    df_stadiums.info()

except requests.exceptions.HTTPError as err:
    print(f"❌ Error HTTP al obtener la página: {err}")
except Exception as e:
    print(f"❌ Error al procesar las tablas: {e}")

Paso 1: Realizando la solicitud HTTP con User-Agent...
Paso 2: Solicitud exitosa (Status 200). Leyendo tablas...
✅ ¡DataFrame extraído con éxito!

--- Primeras 5 filas ---
   Rank                                 Stadium Seating capacity     Region  \
0     1           Rungrado 1st of May Stadium ♦       114,000[1]  East Asia   
1     2                              Camp Nou ♦        99,354[2]     Europe   
2     3                           FNB Stadium ♦        94,736[3]     Africa   
3     4  New Admin­is­trative Capital Stadium ♦        93,940[4]     Africa   
4     5                       Wembley Stadium ♦        90,000[5]     Europe   

        Country                        City  Images  \
0   North Korea                   Pyongyang     NaN   
1         Spain        Barcelona, Catalonia     NaN   
2  South Africa       Johannesburg, Gauteng     NaN   
3         Egypt  New Administrative Capital     NaN   
4       England             London, England     NaN   

                      

In [7]:
display(df_stadiums)

,Rank,Stadium,Seating capacity,Region,Country,City,Images,Home team(s)
0,1,Rungrado 1st of May Stadium ♦,"114,000[1]",East Asia,North Korea,Pyongyang,NaN,"Korea DPR national football team, Korea DPR wo..."
1,2,Camp Nou ♦,"99,354[2]",Europe,Spain,"Barcelona, Catalonia",NaN,FC Barcelona
2,3,FNB Stadium ♦,"94,736[3]",Africa,South Africa,"Johannesburg, Gauteng",NaN,"South Africa national football team, Kaizer Ch..."
3,4,New Admin­is­trative Capital Stadium ♦,"93,940[4]",Africa,Egypt,New Administrative Capital,NaN,Egypt national football team
4,5,Wembley Stadium ♦,"90,000[5]",Europe,England,"London, England",NaN,England national football team
...,...,...,...,...,...,...,...,...
347,348,Bao'an Stadium,40000,East Asia,China,"Shenzhen, Guangdong",NaN,NaN
348,349,Kunming Tuodong Sports Centre Stadium,40000,East Asia,China,"Kunming, Yunnan",NaN,China national football team
349,350,Taizhou Sports Centre Stadium,40000,East Asia,China,"Taizhou, Zhejiang",NaN,NaN
350,351,Wuhu Olympic Stadium,40000,East Asia,China,"Wuhu, Anhui",NaN,NaN


## Limpieza de datos

In [10]:
import pandas as pd
import re
# NOTA: Debes asegurarte de que el DataFrame 'df_stadiums' (extraído previamente) esté cargado en tu entorno.

# --- 1. Limpieza de la columna 'Stadium' ---
# El carácter '♦' puede aparecer en algunos nombres de estadios.
df_stadiums['Stadium'] = df_stadiums['Stadium'].astype(str).str.replace('♦', '', regex=False).str.strip()

# --- 2. Limpieza de la columna 'Capacity' ---
# 2.1 Eliminar números entre corchetes (ej: [1], [2]...) que representan referencias.
# El patrón r'\[.*?\]' busca cualquier texto dentro de corchetes.
df_stadiums['Seating capacity'] = df_stadiums['Seating capacity'].astype(str).str.replace(r'\[.*?\]', '', regex=True)

# 2.2 Eliminar cualquier carácter de nueva línea '\n' (saltos de línea) y espacios al inicio/final.
df_stadiums['Seating capacity'] = df_stadiums['Seating capacity'].str.replace(r'\n', '', regex=True).str.strip()

# 2.3 Eliminar comas (',') que se usan como separadores de miles para prepararlo para la conversión numérica.
df_stadiums['Seating capacity'] = df_stadiums['Seating capacity'].str.replace(',', '', regex=False)

# 2.4 Convertir la columna 'Capacity' a un tipo numérico (integer), forzando errores a NaN.
df_stadiums['Seating capacity'] = pd.to_numeric(df_stadiums['Seating capacity'], errors='coerce')

# --- 3. Eliminar la columna 'Image' ---
# NOTA: En la tabla de Wikipedia, esta columna se llama 'Image/Notes' o similar en el DataFrame.
# La nombraremos según la salida de pd.read_html: 'Image/Notes'
if 'Image/Notes' in df_stadiums.columns:
    df_stadiums = df_stadiums.drop(columns=['Image/Notes'])
elif 'Image' in df_stadiums.columns:
    # Si por alguna razón la columna se llama 'Image'
    df_stadiums = df_stadiums.drop(columns=['Image'])
    
# --- 4. Mostrar el DataFrame limpio ---
print("✅ ¡Limpieza de datos completada!")
print("\n--- Primeras 5 filas del DataFrame limpio ---")
print(df_stadiums.head())
print("\n--- Información del DataFrame (verificar tipo de 'Capacity') ---")
df_stadiums.info()

✅ ¡Limpieza de datos completada!

--- Primeras 5 filas del DataFrame limpio ---
   Rank                               Stadium  Seating capacity     Region  \
0     1           Rungrado 1st of May Stadium          114000.0  East Asia   
1     2                              Camp Nou           99354.0     Europe   
2     3                           FNB Stadium           94736.0     Africa   
3     4  New Admin­is­trative Capital Stadium           93940.0     Africa   
4     5                       Wembley Stadium           90000.0     Europe   

        Country                        City  Images  \
0   North Korea                   Pyongyang     NaN   
1         Spain        Barcelona, Catalonia     NaN   
2  South Africa       Johannesburg, Gauteng     NaN   
3         Egypt  New Administrative Capital     NaN   
4       England             London, England     NaN   

                                        Home team(s)  
0  Korea DPR national football team, Korea DPR wo...  
1         

In [11]:
display(df_stadiums)

,Rank,Stadium,Seating capacity,Region,Country,City,Images,Home team(s)
0,1,Rungrado 1st of May Stadium,114000.0,East Asia,North Korea,Pyongyang,NaN,"Korea DPR national football team, Korea DPR wo..."
1,2,Camp Nou,99354.0,Europe,Spain,"Barcelona, Catalonia",NaN,FC Barcelona
2,3,FNB Stadium,94736.0,Africa,South Africa,"Johannesburg, Gauteng",NaN,"South Africa national football team, Kaizer Ch..."
3,4,New Admin­is­trative Capital Stadium,93940.0,Africa,Egypt,New Administrative Capital,NaN,Egypt national football team
4,5,Wembley Stadium,90000.0,Europe,England,"London, England",NaN,England national football team
...,...,...,...,...,...,...,...,...
347,348,Bao'an Stadium,40000.0,East Asia,China,"Shenzhen, Guangdong",NaN,NaN
348,349,Kunming Tuodong Sports Centre Stadium,40000.0,East Asia,China,"Kunming, Yunnan",NaN,China national football team
349,350,Taizhou Sports Centre Stadium,40000.0,East Asia,China,"Taizhou, Zhejiang",NaN,NaN
350,351,Wuhu Olympic Stadium,40000.0,East Asia,China,"Wuhu, Anhui",NaN,NaN


In [14]:
from azure.storage.filedatalake import DataLakeServiceClient
import pandas as pd
import io
import datetime

# Función para conectarse al servicio de Azure Data Lake usando la connection string
def connect_to_datalake(connection_string):
    try:
        # Crear una instancia del cliente de DataLake usando la connection string
        datalake_service_client = DataLakeServiceClient.from_connection_string(connection_string)
        return datalake_service_client
    except Exception as e:
        print(f"Error al conectarse a Azure Data Lake: {e}")
        return None

# Cargar el DataFrame a Azure Data Lake
def upload_dataframe_to_datalake(df, datalake_service_client, filesystem_name, file_path):
    try:
        # Acceder al filesystem de Azure Data Lake
        filesystem_client = datalake_service_client.get_file_system_client(filesystem_name)
        
        # Crear o obtener el archivo
        file_client = filesystem_client.get_file_client(file_path)
        
        # Convertir el DataFrame a CSV
        csv_data = df.to_csv(index=False)

        # Subir el CSV a Azure Data Lake
        file_client.upload_data(csv_data, overwrite=True)
        print(f"El archivo ha sido cargado correctamente a {file_path}.")
    except Exception as e:
        print(f"Error al cargar el DataFrame a Data Lake: {e}")

# Parámetros de conexión
current_date = datetime.datetime.now().strftime('%Y-%m-%d')

connection_string = "DefaultEndpointsProtocol=https;AccountName=footballdata2004;AccountKey=/BKJz4zQ+8X+XqqArCTv3SLSkDbdIzPz9RXt+298jolnPoVYPuXw1XLcDc0wC01V+pVwfJvLafdq+AStENMpeg==;EndpointSuffix=core.windows.net"
filesystem_name = "footballdata2004"  # El contenedor de tu Data Lake
file_path = f"data/{current_date}/stadiums.csv"  # La ruta dentro del contenedor donde guardarás el archivo

# Conectar al DataLake
datalake_service_client = connect_to_datalake(connection_string)

if datalake_service_client:
    # Cargar el DataFrame a Azure Data Lake
    upload_dataframe_to_datalake(df_stadiums, datalake_service_client, filesystem_name, file_path)

El archivo ha sido cargado correctamente a data/2025-12-08/stadiums.csv.
